# Classification - TSST vs. fTSST (Talk Only)

## Imports and Helper Functions

In [ ]:
import warnings

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import GroupKFold

from fau_colors import cmaps

import biopsykit as bp
from biopsykit.classification.model_selection import SklearnPipelinePermuter
from biopsykit.classification.utils import prepare_df_sklearn

from stresspose_analysis.classification.utils import flatten_wide_format_column_names
from stresspose_analysis.classification.hyperparameter_search import (
    get_model_dict,
    get_hyper_para_dict,
    get_hyper_search_dict,
)

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (8, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

## Setup Paths

In [ ]:
input_path = Path("../../../feature_export/motion_features")
output_path = Path("../../../output/classification/detailed")
feature_path = input_path.joinpath("feature_export/motion_features")
output_path.mkdir(exist_ok=True)

## Load Features and Prepare DataFrame for Sklearn

In [ ]:
feature_path = feature_path.joinpath("motion_features_cleaned.csv")
data = bp.io.load_long_format_csv(feature_path)

In [ ]:
levels_unstack = list(data.index.names)
for level in ["subject", "condition"]:
    levels_unstack.remove(level)
data_wide = data["data"].unstack(levels_unstack)

data_wide = flatten_wide_format_column_names(data_wide)
data_wide.head()

In [ ]:
X, y, groups, group_keys = prepare_df_sklearn(data_wide, label_col="condition", print_summary=True)

## Specify Estimator Combinations and Parameters for Hyperparameter Search

In [ ]:
num_subjects = len(group_keys)
model_dict = get_model_dict()
params_dict = get_hyper_para_dict(num_subjects)
hyper_search_dict = get_hyper_search_dict()

In [ ]:
model_dict

## Setup PipelinePermuter and Cross-Validations for Model Evaluation

In [ ]:
pipeline_permuter = SklearnPipelinePermuter(model_dict, params_dict, hyper_search_dict, random_state=0)

Fit all pipelines

In [ ]:
outer_cv = GroupKFold(5)
inner_cv = GroupKFold(5)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    pipeline_permuter.fit(X, y, outer_cv=outer_cv, inner_cv=inner_cv, groups=groups, use_cache=False)

In [ ]:
data.to_csv(output_path.joinpath(f"{feature_path.stem}_5min.csv"))
pipeline_permuter.to_pickle(output_path.joinpath(f"classification_cumulative_time_5min_pipeline_permuter.pkl"))
